- NOTE!: HERE We ASSUME that the datasets contains already S21, NOT the output power. We keep track of input power in each the dataset name.

Libraries and functions found on web to display content of a .h5 file

In [ ]:
import h5py
import numpy as np
from h5utils import *
import matplotlib.pyplot as plt
import sys

In [ ]:
filename = "03_03_4-8Ghz_50pts.h5";   # Change HERE!!!

Here you can see the content of the file

In [ ]:
"""PRINT ALL the .h5 file directories"""
h5dump(filename,group = "/")

In [ ]:
"""Get simulation parameters"""
params_names, params_values = getDatasetValue(filename,"/",0)

# Print [Check if the order of paramerers is the same of the assegnation below!]
for i in np.arange(0,len(params_names)):
    print(str(params_names[i].decode("utf-8")),": ",str(params_values[i].decode("utf-8")))

# Vna
powerdBm = params_values[0] #[dBm]
navgs = params_values[3]
# Pump
freq_start = float(params_values[1])/1e9 #[GHz]
freq_stop = float(params_values[2])/1e9 #[Ghz]
ampl_min = float(params_values[3]) #[dBm]
ampl_max = float(params_values[4]) #[dBm]
# Bias
volt_min = float(params_values[5]) #[V]
volt_max = float(params_values[6]) #[V]
R = float(params_values[len(params_values)-1])
# You could retrieve current from resistance

# Points densities
n_freq = int(params_values[7])
n_ampl = int(params_values[8])
n_volt = int(params_values[9])

freqs = np.linspace(freq_start, freq_stop, n_freq) 
ampls = np.linspace(ampl_min, ampl_max, n_ampl)
volts = np.linspace(volt_min, volt_max, n_volt)
#volts = [0.227, 0.277, 0.327, 0.377, 0.427]

"""Print parameters arrays"""
print("\n\nIndex:\n")
temp_str = ""
volt_cifers = 3
for i in np.arange(0,n_volt):
    temp_str = temp_str + "[" + str(i) + "] " + str(round(volts[i], volt_cifers)) + "; "
print("Bias Voltage (V): ",temp_str)

temp_str = ""
freq_cifers = 3
for i in np.arange(0,n_freq):
    temp_str = temp_str + "[" + str(i) + "] " + str(round(freqs[i], freq_cifers)) + "; "
print("Pump frequency (GHz): ",temp_str)

temp_str = ""
ampl_cifers = 3
for i in np.arange(0,n_ampl):
    temp_str = temp_str + "[" + str(i) + "] " + str(round(ampls[i], ampl_cifers)) + "; "
print("Pump power (dBm): ",temp_str)

In [ ]:
"""Plot (and get) a specific measure"""
dc_volt_index = 19
pump_freq_index = 0
pump_pow_index = 0


group = "/dc_" + str(dc_volt_index) + "/pumpfreq_" + str(pump_freq_index) + "/pumppow_" + str(pump_pow_index)

fmin = 1e9
fmax = 14e9

#group = "Off"

print("Simulation parameters: ")
print("Bias voltage: ", str(round(volts[dc_volt_index], volt_cifers)), " V")
print("Pump frequency: ", str(round(freqs[pump_freq_index], freq_cifers)), " GHz")
print("Pump power: ", str(round(ampls[pump_pow_index], ampl_cifers)), "dBm")
fig, ax = plotDatasetSpectrum(filename, group, 0, fmin, fmax)
xs, ys = getDatasetValue(filename, group, 0)

xoff, yoff = getDatasetValue(filename,"Off",0)

fig2, ax2 = plt.subplots()
ax2.plot(xs,ys-yoff)
ax2.set_title("Difference between Off measure")

In [ ]:
"""Check Bias and Pump OFF pre and after"""

fig, ax = plt.subplots()
xs, yoff1 = getDatasetValue(filename,"Off",0)
xs, yoff2 = getDatasetValue(filename,"Off_end",0)
ax.plot(xs[100:220]/1e9,yoff2[100:220]-yoff1[100:220])
ax.set_title("Difference between ALL OFF measurements\ntaken at the beginning and at the end")
ax.set_xlabel("frequency [GHz]")
ax.set_ylabel("gain [dBm]")

NOTE!: Both 'Off' measurement and 'On' measurements need to be present in the SAME .h5 file and must be taken adopting the same parameters for the VNA!

- Consider many selected measurements (all the ones associated to the chosen parameters)
- Subtract the 'Off' measure
- Plot all those measurements on the same canvas in the desired span [ fmin,fmax ]

In [ ]:
"""Gain of some selected experiments"""
voltage_indexes = [0]
frequency_indexes = [0,5,10,19]
power_indexes = [0]

fmin = 5e9 #[GHz]
fmax = 8e9 #[GHz]
fspan = fmax - fmin


fig, ax = plt.subplots()
# Get 'Off' measurement (will be subtracted to all measurements)
xoff, yoff = getDatasetValue(filename, "/Off", 0)
# Check if the frequency span [fmin,fmax] is valid
if xoff[0] <= fmin and xoff[len(xoff)-1] >= fmax:
    print("Band: [",round(fmin/1e9,freq_cifers),", ",round(fmax/1e9,freq_cifers),"] GHz")
else:
    print("Invalid frequency span!")
    exit()
# Get i_min, i_max for the defined band
df = xoff[1] - xoff[0]
i_min = round((fmin - xoff[0])/df)
i_max = round(len(xoff) - 1 - (xoff[len(xoff)-1] - fmax)/df)
n_pts = i_max - i_min

for k in voltage_indexes:
    for j in power_indexes:
        for i in frequency_indexes:
            # Get desired dataset
            group = "/dc_" + str(k) + "/pumpfreq_" + str(i) + "/pumppow_" + str(j)
            print(group)
            temp_xs, temp_ys = getDatasetValue(filename, group, 0)
            # Remove the 'Off' measurement
            temp_ys = temp_ys - yoff

            # Focus on the defined BAND
            xs = temp_xs[i_min:i_max]
            ys = temp_ys[i_min:i_max]

            # Plot S21 in the desired Fspan and get 'gain band average'
            ax.plot(xs/1e9,ys,label=(str(round(volts[k],volt_cifers))+"V,"+str(round(ampls[j],ampl_cifers))+"dBm,"+str(round(freqs[i],freq_cifers))+"GHz, pts"+str(n_pts)))

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.grid()
ax.set_xlabel("Signal frequency [GHz]")
ax.set_ylabel("S21")
ax.set_title("Paramp gain\n['Off' measurement subtracted]")

Find optimal parameters among *ALL* the measurements in the .h5 file

In [ ]:
"""BEST PARAMETERS"""

fmin = 5e9 #[GHz]
fmax = 8e9 #[GHz]
fspan = fmax - fmin


# Get 'Off' measurement (will be subtracted to all measurements)
#xoff, yoff = getDatasetValue(filename, "/Off", 0)
group = "/dc_" + "0" + "/pumpfreq_" + "0" + "/pumppow_" + "0"
#xoff, yoff = getDatasetValue(filename, group, 0)
#yoff = np.zeros(len(yoff))

# Check if the frequency span [fmin,fmax] is valid
if xoff[0] <= fmin and xoff[len(xoff)-1] >= fmax:
    print("Band: [",round(fmin/1e9,freq_cifers),", ",round(fmax/1e9,freq_cifers),"] GHz")
else:
    sys.exit("Invalid frequency span!")
# Get i_min, i_max for the defined band
df = xoff[1] - xoff[0]
i_min = round((fmin - xoff[0])/df)
i_max = round(len(xoff) - 1 - (xoff[len(xoff)-1] - fmax)/df)
n_pts = i_max - i_min

band_averages = np.zeros((len(volts),len(freqs),len(ampls)))
max_average = [0,0,0]    # Indexes of params for best band average
for k in np.arange(0,len(volts)):
    for j in np.arange(0,len(ampls)):
        for i in np.arange(0,len(freqs)):
            # Get desired dataset
            group = "/dc_" + str(k) + "/pumpfreq_" + str(i) + "/pumppow_" + str(j)
            try:
                temp_xs, temp_ys = getDatasetValue(filename, group, 0)
                # Remove the 'Off' measurement
                temp_ys = temp_ys - yoff
                # Focus on the defined BAND
                xs = temp_xs[i_min:i_max]
                ys = temp_ys[i_min:i_max]

                band_averages[k,i,j] = np.mean(ys)
                if band_averages[max_average[0],max_average[1],max_average[2]] < band_averages[k,i,j]:
                    max_average = [k,i,j]
            except:
                print(i)

# Get params for maximum band average
best_voltage = volts[max_average[0]]
best_freq = freqs[max_average[1]]
best_pow = ampls[max_average[2]]
print("Best params: ",best_voltage,"V, ",best_freq,"GHz, ",best_pow,"dBm")
print("Best Band average gain: ",round(band_averages[max_average[0],max_average[1],max_average[2]],3),"dB")

# Plot band average varying only Pump frequency (other two params fixed at 'optimal' values)
fig_freq, ax_freq = plt.subplots()
ax_freq.plot(freqs, band_averages[max_average[0],:,max_average[2]],'.-')
ax_freq.set_title("Band average ["+str(round(fmin/1e9,freq_cifers))+", "+str(round(fmax/1e9,freq_cifers))+"] GHz\nBias: "+str(round(best_voltage,volt_cifers))+"V; Pump Pow: "+str(round(best_pow,ampl_cifers))+"dBm")
ax_freq.set_xlabel("Pump frequency [GHz]")
ax_freq.set_ylabel("Averaged gain [dB]")

# Plot band average varying only Pump power (other two params fixed at 'optimal' values)
fig_pow, ax_pow = plt.subplots()
ax_pow.plot(ampls, band_averages[max_average[0],max_average[1],:],'.-')
ax_pow.set_title("Band average ["+str(round(fmin/1e9,freq_cifers))+", "+str(round(fmax/1e9,freq_cifers))+"] GHz\nBias: "+str(round(best_voltage,volt_cifers))+"V; Pump Freq: "+str(round(best_freq,freq_cifers))+"GHz")
ax_pow.set_xlabel("Pump power [dBm]")
ax_pow.set_ylabel("Averaged gain [dB]")

# Plot band average varying only Bias voltage (other two params fixed at 'optimal' values)
fig_freq, ax_freq = plt.subplots()
ax_freq.plot(volts, band_averages[:,max_average[1],max_average[2]],'.-')
ax_freq.set_title("Band average ["+str(round(fmin/1e9,freq_cifers))+", "+str(round(fmax/1e9,freq_cifers))+"] GHz\nPump Freq: "+str(round(best_freq,freq_cifers))+"GHz; Pump Pow: "+str(round(best_pow,ampl_cifers))+"dBm")
ax_freq.set_xlabel("Bias voltage [V]")
ax_freq.set_ylabel("Averaged gain [dB]")

In [ ]:
"""DELETE a dataset"""
full_dataset_path = "/group/dataset_to_delete"
deleteDataset(filename, full_dataset_path)

In [ ]:
"""Save a dataset"""
filename_save = "save.h5"
full_dataset_path = "/group/datasetname"

mat = [xs1, ys1]
hf = h5py.File(filename_save, 'a')
hf.create_dataset(full_dataset_path, data = mat)
hf.close()